# **Webscrapping and Analysis of Global COVID-19 Pandemic Data**

**Author:** ***`Ayobami Yusuf`***

### **Project Overview**:
In this project, I will be scraping **Covid-19 data** off of ***Wikipedia*** using **R's** ***`rvest`*** ***library*** and conducting pretty basic exploratory data analysis (EDA).<p>
This project has been divided into two sections:
### **1. Data Wrangling (Webscrappping, Data Cleaning, and Data Export)**
### **2. Exploratory Data Analysis**

It is worthy of note, however, that the main focus of this project is on **ETL (Extract, Tranform and Load) processes**, not **data analysis**. To complete this project, I will carry out 5 predetermined steps/tasks.

### Package Installation and Loading

In [1]:
#install httr and rvest - r's webscrapping library
install.packages("httr")
install.packages("rvest")

#next, I load the intsalled packages
require("httr")
require("rvest")

library(httr)
library(rvest)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: httr

Loading required package: rvest



> ***Now that the packages have been installed and loaded into the notebook environment, I can proceed to undertake each task necessary to complete the project.*** <p> 
***Each task will be be highlighted, and if needed, a brief description of expected deliverable and actions needing clarity will be briefly described.***

# **Data Wrangling**

## Task 1: Accessing the COVID-19 Pandemic Wikipedia Page using HTTP `request`
#### The goal of this task is to get the HTML page containing the Covid-19 data using HTTP `request` (httr library)
> The target website is **[here]( https://en.wikipedia.org/w/index.php?title=Template:COVID-19_testing_by_country)** and it contains two parts: 
1. The base URL: [https://en.wikipedia.org/w/index.php](`https://en.wikipedia.org/w/index.php`)
2. The URL parameter: `title=Template:COVID-19_testing_by_country`, seperated by question mark ?

In [2]:
#I use the GET function, in the httr library with a url argument to get a HTTP response
url <- "https://en.wikipedia.org/w/index.php?title=Template:COVID-19_testing_by_country"

#then I use the return function to return the response
response <- GET(url)
response

Response [https://en.wikipedia.org/w/index.php?title=Template:COVID-19_testing_by_country]
  Date: 2022-08-18 08:52
  Status: 200
  Content-Type: text/html; charset=UTF-8
  Size: 415 kB
<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Template:COVID-19 testing by country - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames...
"CS1 German-language sources (de)","CS1 Azerbaijani-language sources (az)","C...
"CS1 uses Japanese-language script (ja)","CS1 Japanese-language sources (ja)"...
"COVID-19 pandemic templates"],"wgPageContentLanguage":"en","wgPageContentMod...
true,"wgGETopicsMatchModeEnabled":false,"wgGEStructuredTaskRejectionReasonTex...
...

## TASK 2: Extracting COVID-19 Testing Data Table from the Wiki HTML Page
#### The goal of this task is to extract the testing data table on the page and convert it into a R data frame. To achieve this;
1. I use the `read_html` function in rvest library to get the root html node from response
2. Then I get the first table in the HTML root node using the `html_node` function
3. Lastly, I read the table node as a data frame using `html_table` function

In [3]:
#I get the root html node from the http response in task 1 
root_node <- read_html(response)

#Next, I get the table node from the root html node
table_node <- html_node(root_node, "table")

#lastly, I read the table node and convert it into a R data frame, and print the the first few rows of the data frame for review
covid_df <- html_table(table_node)
head(covid_df)

Country or region,Date[a],Tested,Units[b],Confirmed(cases),"Confirmed /tested,%","Tested /population,%","Confirmed /population,%",Ref.
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Afghanistan,17 Dec 2020,"154,767",samples,"49,621",32.1,0.40,0.13,[1]
Albania,18 Feb 2021,"428,654",samples,"96,838",22.6,15.0,3.4,[2]
Algeria,2 Nov 2020,"230,553",samples,"58,574",25.4,0.53,0.13,[3][4]
Andorra,23 Feb 2022,"300,307",samples,"37,958",12.6,387,49.0,[5]
Angola,2 Feb 2021,"399,228",samples,"20,981",5.3,1.3,0.067,[6]
Antigua and Barbuda,6 Mar 2021,"15,268",samples,832,5.4,15.9,0.86,[7]


## TASK 3: Pre-processing and Exporting the Extracted Dataframe
#### The goal of this task is to pre-process the extracted data frame, and export it as a csv file. **"Pre-process"** here is synonymous to **"Data Cleaning"**, a phrase more widely used within the Data Ecosystem.
#### Let's get a high-level overwiew of the dataframe first. That will give us some insights into what cleaning tasks need to be carried out.

In [4]:
summary(covid_df)

 Country or region    Date[a]             Tested            Units[b]        
 Length:173         Length:173         Length:173         Length:173        
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  
 Confirmed(cases)   Confirmed /tested,% Tested /population,%
 Length:173         Length:173          Length:173          
 Class :character   Class :character    Class :character    
 Mode  :character   Mode  :character    Mode  :character    
 Confirmed /population,%     Ref.          
 Length:173              Length:173        
 Class :character        Class :character  
 Mode  :character        Mode  :character  

First thing we see from the summary output is that the column names are too vague to understand. It is best practice to have descriptive column names for easy reference. We also have the issue of incorrect data types with some of the columns - for example, the **`Tested`** column shows as character, which should be a numeric column.

As such, the data frame read from HTML table will need some pre-processing (cleaning) such as **removing irrelvant columns, renaming columns, and converting columns into proper data types**.

In [7]:
#I define a function that does the cleaning tasks in one breeze
clean_covid_df <- function(df) {
    
    shape <- dim(df)

    #this removes the World row
    df<-df[!(df$`Country or region`=="World"),]
    #this removes the last row
    df <- df[1:172, ]
    
    #I dont need the Units and Ref columns, so they can be removed
    df["Ref."] <- NULL
    df["Units[b]"] <- NULL
    
    #Renaming the columns
    names(df) <- c("country", "date", "tested", "confirmed", "confirmed.tested.ratio", "tested.population.ratio", "confirmed.population.ratio")
    
    #This converts columns data types into proper ones
    df$country <- as.factor(df$country)
    df$date <- as.factor(df$date)
    df$tested <- as.numeric(gsub(",","",df$tested))
    df$confirmed <- as.numeric(gsub(",","",df$confirmed))
    df$'confirmed.tested.ratio' <- as.numeric(gsub(",","",df$`confirmed.tested.ratio`))
    df$'tested.population.ratio' <- as.numeric(gsub(",","",df$`tested.population.ratio`))
    df$'confirmed.population.ratio' <- as.numeric(gsub(",","",df$`confirmed.population.ratio`))
    
    return(df)
}

#### Now, let's call the clean_covid_df function and assign it to a "clean_df" dataframe.

In [8]:
#Calling the `clean_covid_df` function and assign it to a new data frame
clean_df <- clean_covid_df(covid_df)
head(clean_df)

country,date,tested,confirmed,confirmed.tested.ratio,tested.population.ratio,confirmed.population.ratio
<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Afghanistan,17 Dec 2020,154767,49621,32.1,0.40,0.130
Albania,18 Feb 2021,428654,96838,22.6,15.00,3.400
Algeria,2 Nov 2020,230553,58574,25.4,0.53,0.130
Andorra,23 Feb 2022,300307,37958,12.6,387.00,49.000
Angola,2 Feb 2021,399228,20981,5.3,1.30,0.067
Antigua and Barbuda,6 Mar 2021,15268,832,5.4,15.90,0.860


#### Let's get a high-level overview of the dataframe (the cleaned one this time around) to observe changes made

In [9]:
summary(clean_df)

                country             date         tested         
 Afghanistan        :  1   13 Aug 2022:  9   Min.   :     3880  
 Albania            :  1   3 Aug 2022 :  5   1st Qu.:   512037  
 Algeria            :  1   4 Aug 2022 :  4   Median :  3029859  
 Andorra            :  1   1 Mar 2021 :  3   Mean   : 30695088  
 Angola             :  1   15 Aug 2022:  3   3rd Qu.: 11827419  
 Antigua and Barbuda:  1   23 Jul 2021:  3   Max.   :929349291  
 (Other)            :166   (Other)    :145                      
   confirmed        confirmed.tested.ratio tested.population.ratio
 Min.   :       0   Min.   : 0.00          Min.   :   0.0065      
 1st Qu.:   37636   1st Qu.: 5.00          1st Qu.:   9.3750      
 Median :  281196   Median :10.15          Median :  42.3500      
 Mean   : 2410938   Mean   :11.45          Mean   : 166.7915      
 3rd Qu.: 1207072   3rd Qu.:15.43          3rd Qu.: 139.2500      
 Max.   :90749469   Max.   :63.80          Max.   :3024.0000      
           

After cleaning, we can see the columns are now simplified with more descriptive column names, and columns types are converted into correct types.

**The data frame has following columns:**

- country - *The name of the country*
- date - *Reported date*
- tested - *Total tested cases by the reported date*
- confirmed - *Total confirmed cases by the reported date*
- confirmed.tested.ratio - *The ratio of confirmed cases to the tested cases*
- tested.population.ratio - *The ratio of tested cases to the population of the country*
- confirmed.population.ratio - *The ratio of confirmed cases to the population of the country*

Now, we can call the `write.csv()` function to save the dataframe to a csv file.

In [10]:
#Exporting the dataframe to a csv file
write.csv(clean_df, file = "covid-19_df.csv", row.names = FALSE)

# **Exploratory Data Analysis (EDA)**

## Task 4: Calculating Worldwide COVID-19 test-positive ratio.
#### The goal of this task is to get the total confirmed and tested cases worldwide, and try to figure out the overall positive ratio using the simple confirmed cases / tested cases formula

In [16]:
#Gets the total confirmed cases worldwide
confirmed_cases <- clean_df[ , 4]
total_confirmed_cases <- sum(confirmed_cases)
print(paste("Total confirmed cases:", total_confirmed_cases))

#Gets the total tested cases worldwide
total_cases <- clean_df[ , 3]
total_tested_cases <- sum(total_cases)
print(paste("Total tested cases:", total_tested_cases))

#Gets the positive ratio (confirmed / tested)
positive_ratio <- total_confirmed_cases/total_tested_cases
print(paste("Positive ratio:", positive_ratio*100))

[1] "Total confirmed cases: 414681419"
[1] "Total tested cases: 5279555099"
[1] "Positive ratio: 7.8544765841831"


Based on the number of people tested globally and the number of confirmed cases, **the global average positive cases ratio is just a little under 8%**. Essentially, about **8 out every 100** people **tested** were **positive**. Depending on the context though, maybe the Covid-19 pandemic isn't that much of a "PANDEMIC" - no disregard to the lives lost. 

## Task 5: Which Countries were Dealt the Hugest Blow?
#### The main task here is to discover the countries that felt the largest impact of the Covid-19 pandemic. This is assessed by comparing their confimed.population.ratio

In [18]:
sorted_df <-clean_df[order(-clean_df$confirmed.population.ratio), ]
high_impact_countries <- head(sorted_df, 10)
high_impact_countries[c(1,7)]

country,confirmed.population.ratio
<fct>,<dbl>
Cyprus[d],65.8
Faroe Islands,65.7
San Marino,58.5
Denmark[e],55.9
Iceland,55.8
Austria,54.9
Slovenia,52.7
Andorra,49.0
Switzerland[l],46.1


Looking at the table above, many analysts may argue that the reason the ratio for the above countries are high is because most of them have low population, but that is exactly the idea. The rate of exposure is too high too for these countries despite their low population. Imagine having about **65%** of an entire population living with a particular virus when the global average infection rate is just below **8%**. That's scary. If I were living in any of these country and I hear about a new global outbreak of a virus (which we pray against), I would leave such country the very minute. And I am leaving with my human and physical capital investments, if possible.

# **Summary:**
This project was embarked upon to put to practice the knowledge and skills I gained while taking the **[Introduction to R for Data Science](https://www.coursera.org/learn/introducton-r-programming-data-science)** course curated by **[Coursera](https://www.coursera.org/)**. Working on a real-world project seemed like a good way to put on my Data Analyst hat and try to solve an actual problem. 
Special thanks to **[Data Professor](https://www.youtube.com/c/DataProfessor)** for his tutorial on how to **[Use native R on Google Colab for Data Science]((https://www.youtube.com/watch?v=huAWa0bqxtA)**.

My name is Ayobami Yusuf, I am a life-long learner devoted to Data Science, Machine Learning, Data Analytics, and helping Individuals and Businesses stay competitive by leveraging their data and modern data stack and tools. You can connect with me on **[LinkedIn](www.linkedin.com/in/ayobami-yusuf)**. 